### Installing packages and pulling the model
This cell runs shell commands that install the MONAI package (where all models are stored), then download the lung_nodule_ct_detection model specifically.  The model and its accompanying files will be downloaded to the /bundles/lung_nodle_ct_detection/ subdirectory.

In [ ]:
!pip install monai[fire]
!python -m monai.bundle download "lung_nodule_ct_detection" --bundle_dir "bundles/"  # model and accompanying files will be in the /bundles folder

2026-02-17 11:23:40,083 - INFO - --- input summary of monai.bundle.scripts.download ---
2026-02-17 11:23:40,083 - INFO - > name: 'lung_nodule_ct_detection'
2026-02-17 11:23:40,083 - INFO - > bundle_dir: 'bundles/'
2026-02-17 11:23:40,083 - INFO - > source: 'monaihosting'
2026-02-17 11:23:40,083 - INFO - > remove_prefix: 'monai_'
2026-02-17 11:23:40,083 - INFO - > progress: True
2026-02-17 11:23:40,083 - INFO - ---





Fetching 20 files: 100%|██████████| 20/20 [00:00<00:00, 1849.91it/s]


----------------------------------------
### Loading the model
MONAI provides a "config" file that tells you how to work with the model.  Specifies:
1. Model architecture
2. Data preprocessing specifications
3. Inference logic - the model wants a patch of the 3D image of size 512x512x192
4. Output format - how the model will output its prediction (coordinates of a bounding box surrounding the tumor)

So, what we will do is first parse the config file and get this important info.  Then, we can build the architecture specified in the config file to instantiate the model.  Finally, we load the trained weights into the architecture we instantiated.

In [20]:
import os
import torch
from monai.bundle import ConfigParser

# !!! YOU NEED TO CHANGE THIS TO YOUR ROOT PATH !!!
bundle_root = os.path.join(os.getcwd(), 'bundles\\lung_nodule_ct_detection')

#  defines paths to the bundle and config file
config_file = os.path.join(bundle_root, "configs", "inference.json")
model_path = os.path.join(bundle_root, "models", "model.pt")

# parses the config file
parser = ConfigParser()
parser.read_config(config_file)

# defines a new element in the parsed config file - the path to our bundle folder
parser["bundle_root"] = bundle_root

#print(parser)  # see the dictionary inside if you want

In [21]:
# instantiates the network - ie, creates a skeleton architecture as described in the config file
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network = parser.get_parsed_content("network_def").to(device)

# loads the trained weights into the skeleton architecture
network.load_state_dict(torch.load(model_path, map_location=device))
network.eval()

# load the preprocessing transforms
# this is a series of transforms that loads the image, confirms shape, normalizes, etc
preprocessing = parser.get_parsed_content("preprocessing")

print("Model and preprocessing pipeline ready.")
#print(preprocessing)

Model and preprocessing pipeline ready.


monai.transforms.spatial.dictionary Orientationd.__init__:labels: Current default value of argument `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` was changed in version None from `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` to `labels=None`. Default value changed to None meaning that the transform now uses the 'space' of a meta-tensor, if applicable, to determine appropriate axis labels.


--------------------------------
### Investigating the model
We need to be aware of the actual architecture we're working with, number of parameters, etc.  These cells will output information on this.  We will first look at the named sub-networks that are within the model.  These are:
1. feature_extractor: extracts info from the scan, turning the raw input into meaningful features
2. classification_head: this will look at the extracted features and classify "is there a nodule/tumor"
3. regression_head: this will actually define a bounding box around the nodule/tumor, "where exactly is the nodule"

We'll look at the parameters and layers in each.

In [22]:
# the model has named sub-networks within - the feature_extractor, classification_head, and regression_head.  
for name, child in network.named_children():
    print(name)

total_params = sum(p.numel() for p in network.parameters())
print(f"\nTotal Parameters in the entire Lung Nodule Detector: {total_params:,}")

feature_extractor
classification_head
regression_head

Total Parameters in the entire Lung Nodule Detector: 20,902,741


In [23]:
# iterates through the sub-networks and shows how many parameters are in each
def print_detection_summary(model):
    print(f"{'Component':<25} | {'Type':<20} | {'Parameters':<15}")
    print("-" * 65)
    
    # 2. Feature Extractor (FPN)
    params = sum(p.numel() for p in model.feature_extractor.parameters())
    print(f"{'Feature Extractor':<25} | {'FPN':<20} | {params:,}")
    
    # 3. Detection Heads
    # RetinaNet has a class_head and a box_head
    params_class = sum(p.numel() for p in model.classification_head.parameters())
    params_box = sum(p.numel() for p in model.regression_head.parameters())
    print(f"{'Classification Head':<25} | {'Sub-network':<20} | {params_class:,}")
    print(f"{'Regression Head':<25} | {'Sub-network':<20} | {params_box:,}")
    
    total_params = sum(p.numel() for p in model.parameters())
    print("-" * 65)
    print(f"{'TOTAL':<25} | {'':<20} | {total_params:,}")

print_detection_summary(network)

Component                 | Type                 | Parameters     
-----------------------------------------------------------------
Feature Extractor         | FPN                  | 6,595,648
Classification Head       | Sub-network          | 7,101,699
Regression Head           | Sub-network          | 7,205,394
-----------------------------------------------------------------
TOTAL                     |                      | 20,902,741


In [24]:
# To see the types of layers in the feature extractor:
print("--- Feature Extractor ---")
print(network.feature_extractor)

--- Feature Extractor ---
BackboneWithFPN(
  (body): IntermediateLayerGetter(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 1), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): ResNetBottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [25]:
# To see the Classification Head architecture:
print("--- Classification Head ---")
print(network.classification_head)

# To see the Regression Head architecture:
print("\n--- Regression Head ---")
print(network.regression_head)

--- Classification Head ---
RetinaNetClassificationHead(
  (conv): Sequential(
    (0): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): GroupNorm(8, 256, eps=1e-05, affine=True)
    (2): ReLU()
    (3): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (4): GroupNorm(8, 256, eps=1e-05, affine=True)
    (5): ReLU()
    (6): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (7): GroupNorm(8, 256, eps=1e-05, affine=True)
    (8): ReLU()
    (9): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (10): GroupNorm(8, 256, eps=1e-05, affine=True)
    (11): ReLU()
  )
  (cls_logits): Conv3d(256, 3, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
)

--- Regression Head ---
RetinaNetRegressionHead(
  (conv): Sequential(
    (0): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): GroupNorm(8, 256, eps=1e-05, affine=True)
 

-------------------------
### Running Inference on example data
Now we need to actually load some example data and run it through the model.  From this, we can see how our laptops handle the model, inference time, etc. to start benchmarking.

In [26]:
import os
import glob

def find_ct_series(root_path):
    patient_data = []
    
    # Walk through the LIDC-IDRI directory
    for patient_dir in os.listdir(root_path):
        patient_path = os.path.join(root_path, patient_dir)
        if not os.path.isdir(patient_path):
            continue
            
        # Find all subdirectories that contain .dcm files
        dcm_folders = []
        for root, dirs, files in os.walk(patient_path):
            if any(f.lower().endswith('.dcm') for f in files):
                dcm_count = len([f for f in files if f.lower().endswith('.dcm')])
                dcm_folders.append((root, dcm_count))
        
        # The CT scan is almost always the folder with the MOST files 
        # (Segmentations usually only have 1 or a few files)
        if dcm_folders:
            ct_folder = max(dcm_folders, key=lambda x: x[1])[0]
            patient_data.append({"image": ct_folder, "id": patient_dir})
            
    return patient_data

# Set your root LIDC path
base_path = os.path.join(os.getcwd(), "test_data\\manifest-1771334957344\\LIDC-IDRI")
test_data_list = find_ct_series(base_path)

for p in test_data_list:
    print(f"Patient: {p['id']} | Found CT in: ...{p['image'][-30:]}")

Patient: LIDC-IDRI-0001 | Found CT in: ...-30178\3000566.000000-NA-03192
Patient: LIDC-IDRI-0002 | Found CT in: ...-98329\3000522.000000-NA-04919
Patient: LIDC-IDRI-0003 | Found CT in: ...-94866\3000611.000000-NA-03264
Patient: LIDC-IDRI-0004 | Found CT in: ...-91780\3000534.000000-NA-58228


In [8]:
from monai.data import Dataset, DataLoader

# Use the list we just generated
test_ds = Dataset(data=test_data_list, transform=preprocessing)
test_loader = DataLoader(test_ds, batch_size=1)

# Now you can iterate and run inference without worrying about folder names
for batch in test_loader:
    image_data = batch["image"]
    print(f"Loaded volume for inference with shape: {image_data.shape}")
    # Run detector(image_data) here...
    with torch.no_grad():
        output = network(image_data) 
        print("Detected Boxes:", output)
    

Loaded volume for inference with shape: torch.Size([1, 1, 512, 512, 133])
Detected Boxes: {'classification': [metatensor([[[[[-5.9860, -6.1824, -6.2430,  ..., -6.2426, -6.2227, -6.0696],
           [-6.3184, -6.3630, -6.3818,  ..., -6.3687, -6.3501, -6.2383],
           [-6.3128, -6.3825, -6.4435,  ..., -6.4268, -6.3666, -6.2621],
           ...,
           [-6.3084, -6.3451, -6.4783,  ..., -6.5005, -6.4296, -6.3206],
           [-6.2929, -6.3992, -6.4498,  ..., -6.4838, -6.4557, -6.2799],
           [-6.0103, -6.2404, -6.3427,  ..., -6.3774, -6.3085, -6.1137]],

          [[-6.2973, -6.3777, -6.4000,  ..., -6.3778, -6.3843, -6.2851],
           [-6.5143, -6.4803, -6.5069,  ..., -6.4619, -6.4818, -6.3438],
           [-6.4275, -6.4573, -6.5734,  ..., -6.5475, -6.4907, -6.3728],
           ...,
           [-6.4048, -6.3928, -6.5444,  ..., -6.5745, -6.5502, -6.3909],
           [-6.3910, -6.4084, -6.4842,  ..., -6.5469, -6.5418, -6.4014],
           [-6.2377, -6.2360, -6.3322,  ..., -6.3